In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
data = pd.read_csv("Tokenized_HateSpeechDetection.csv")
X=data['Tokens']
y=data['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train

14622    ['there', 'new', 'shop', 'block', 'with', 'the...
5049                                ['fag', 'are', 'evil']
7386                               ['jizz', 'for', 'jazz']
11513    ['data', 'show', 'different', 'trend', 'for', ...
7765                      ['you', 'lying', 'sack', 'shit']
                               ...                        
11284    ['people', 'who', 'take', 'time', 'out', 'thei...
11964                         ['fucking', 'hate', 'house']
5390        ['ssssssshhhhhhhooooot', 'the', 'bllllllakkk']
860                         ['nigger', 'are', 'redundant']
15795             ['this', 'computer', 'utter', 'rubbish']
Name: Tokens, Length: 14076, dtype: object

In [4]:
from gensim.models import Word2Vec
import numpy as np
def word2vec_embedding_sg(texts):
    model = Word2Vec(sentences=X_train, vector_size=200, window=6, min_count=1, workers=4,sg=1)
    word_vectors = model.wv
    #print(word_vectors)

    def get_word2vec_embeddings(text, word_vectors):
        embeddings = [word_vectors[word] for word in text if word in word_vectors]
        if embeddings:
            return np.mean(embeddings, axis=0)
        else:
            return np.zeros(200)

    embeddings = np.array([get_word2vec_embeddings(text, word_vectors) for text in texts])
    return embeddings
X_train_w2v=word2vec_embedding_sg(X_train)
X_test_w2v=word2vec_embedding_sg(X_test)



# Logistic Regression Model:
Logistic regression is a statistical model used for binary classification tasks. It estimates the probability that a given input belongs to a certain class.

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score

# Train a Logistic Regression model
lr_clf = LogisticRegression()
lr_clf.fit(X_train_w2v, y_train)

# Make predictions
y_pred = lr_clf.predict(X_test_w2v)

# Evaluate the classifier
print("Logistic Regression")
# Precision
precision = precision_score(y_test, y_pred)
print("Precision:", precision)

# Recall
recall = recall_score(y_test, y_pred)
print("Recall:", recall)


# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# ROC-AUC
roc_auc = roc_auc_score(y_test, y_pred)
print("ROC-AUC Score:", roc_auc)

#f1_score
f1 = f1_score(y_test, y_pred)
print("f1:", f1)



Logistic Regression
Precision: 1.0
Recall: 0.001402524544179523
Accuracy: 0.7454545454545455
ROC-AUC Score: 0.7007012622720898
f1: 0.0028011204481792717


In [8]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.01, 0.05, 0.1, 0.5, 1.0, 10.0],
    'solver': ['liblinear', 'lbfgs']
}

# Initialize GridSearchCV
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='accuracy')

# Fit the model
grid_search.fit(X_train_w2v, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Make predictions
y_pred = best_model.predict(X_test_w2v)

# Evaluate the classifier
print("Best Logistic Regression using Grid Search")
print("Precision:", precision_score(y_test, y_pred))
print("f1:", f1_score(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred))



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
    raise ValueError(
ValueError: Solv

Best Logistic Regression using Grid Search
Precision: 0.5986037234042554
f1: 0.7408209291835814
Accuracy: 0.821590909090909
ROC-AUC Score: 0.6884583080490441
